Semantic Segmentation

In [ ]:
! git clone https://github.com/sunghoonYoon/Kitti_TA

In [ ]:
from PIL import Image
from torchvision import transforms
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from kitti_seg import DeepWV3Plus, decode_segmap, denorm

In [ ]:
net = DeepWV3Plus(num_classes=19).cuda()
state_dict= torch.load('/content/drive/MyDrive/ME459_segmentation/cityscapes_best.pth')
net = torch.nn.DataParallel(net)

# print(state_dict)
net.load_state_dict(state_dict['state_dict'],strict=False)

net.eval()

In [ ]:
image= np.asarray(Image.open('/content/drive/MyDrive/ME459_segmentation/kaist_road.jpg').convert('RGB'))
MEAN = [0.45734706, 0.43338275, 0.40058118]
STD = [0.23965294, 0.23532275, 0.2398498]

toTensor = transforms.ToTensor()
normTensor = transforms.Normalize(MEAN,STD)
imageT = normTensor(toTensor(image))

with torch.no_grad():
  for i in [1]:
    imageT = F.interpolate(imageT.unsqueeze(0), size=(512,1024),mode='bilinear',align_corners=True)
    out = net(imageT.cuda())


prob = F.softmax(out,1)
pred = np.argmax(prob[0].detach().cpu().numpy(),axis=0)

segmap = decode_segmap(pred)

plt.imshow(denorm(imageT)[0].permute(1,2,0))
plt.imshow(segmap,alpha=0.7)
plt.show()

Stereo matching

In [ ]:
from kitti_stereo import PSMNet, test_img
from collections import OrderedDict

In [ ]:
max_disp = 192
model = PSMNet(max_disp)
model = model.cuda()
model.eval()

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/ME459_stereo/pretrained_model_KITTI2015.tar')
new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    name = k[7:]
    new_state_dict[name] = v    
model.load_state_dict(new_state_dict)    
normal_mean_var = {'mean': [0.485, 0.456, 0.406],
                    'std': [0.229, 0.224, 0.225]}
infer_transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize(**normal_mean_var)])    

imgL_o = Image.open(left_image_path).convert('RGB')
imgR_o = Image.open(right_image_path).convert('RGB')

imgL = infer_transform(imgL_o)
imgR = infer_transform(imgR_o) 

# pad to width and hight to 16 times
if imgL.shape[1] % 16 != 0:
    times = imgL.shape[1]//16       
    top_pad = (times+1)*16 -imgL.shape[1]
else:
    top_pad = 0

if imgL.shape[2] % 16 != 0:
    times = imgL.shape[2]//16                       
    right_pad = (times+1)*16-imgL.shape[2]
else:
    right_pad = 0    

imgL = F.pad(imgL,(0,right_pad, top_pad,0)).unsqueeze(0)
imgR = F.pad(imgR,(0,right_pad, top_pad,0)).unsqueeze(0)

start_time = time.time()
pred_disp = test_img(imgL,imgR)
if top_pad !=0 or right_pad != 0:
    img = pred_disp[top_pad:,:-right_pad]
else:
    img = pred_disp
plt.figure()
plt.title('left image')
plt.imshow(imgL_o)
plt.figure()
plt.title('disparity output')
plt.imshow(img)
plt.show()
print('time = %.2f' %(time.time() - start_time))